# Week 3 - SQL Corner

https://www.digitalocean.com/community/tutorials/how-to-use-the-sqlite3-module-in-python-3

### Tables

* Students
* Quizzes
* Quiz_Details
* Quiz_Results

In [1]:
import sqlite3

connection = sqlite3.connect('looneylms.db') # sqlite will create looneylms.db if it doesn't exist, 
                                             # a file now in the same folder as this file
cursor = connection.cursor()
print(connection.total_changes)

0


In [2]:
# multiline strings in Python can be sorrounded with ''' Multi line text '''
query = '''
CREATE TABLE people (
    id INTEGER PRIMARY KEY, 
    firstname TEXT, 
    lastname TEXT, 
    year INTEGER
)
'''
cursor.execute(query)

In [3]:
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Bugs', 'Bunny', 1)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Fifi', 'La Fume', 3)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Daffy', 'Duck', 2)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Foghorn', 'Leghorn', 1)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Tweety', 'Bird', 4)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Porky', 'Pig', 4)")

In [4]:
rows = cursor.execute("SELECT * FROM people").fetchall()
print(rows)

[(1, 'Bugs', 'Bunny', 1), (2, 'Fifi', 'La Fume', 3), (3, 'Daffy', 'Duck', 2), (4, 'Foghorn', 'Leghorn', 1), (5, 'Tweety', 'Bird', 4), (6, 'Porky', 'Pig', 4)]


In [5]:
query = '''
CREATE TABLE quizzes (
    id INTEGER PRIMARY KEY, 
    quizname TEXT, 
    description TEXT
)
'''
cursor.execute(query)

In [6]:
cursor.execute("INSERT INTO quizzes(quizname, description) VALUES ('Looney', 'quiz about laughing')")
cursor.execute("INSERT INTO quizzes(quizname, description) VALUES ('Tune', 'quiz about tunes')")

In [7]:
rows = cursor.execute("SELECT * FROM quizzes").fetchall()
print(rows)

[(1, 'Looney', 'quiz about laughing'), (2, 'Tune', 'quiz about tunes')]


In [8]:
query = '''
CREATE TABLE quiz_details (
    id INTEGER PRIMARY KEY, 
    quizzesid INTEGER, 
    items INTEGER
)
'''
cursor.execute(query)

In [9]:
cursor.execute("INSERT INTO quiz_details(quizzesid, items) VALUES (1, 10)")
cursor.execute("INSERT INTO quiz_details(quizzesid, items) VALUES (2, 10)")

In [10]:
rows = cursor.execute("SELECT * FROM quiz_details").fetchall()
print(rows)

[(1, 1, 10), (2, 2, 10)]


In [11]:
query = '''
CREATE TABLE quiz_results (
    id INTEGER PRIMARY KEY, 
    quizzesid INTEGER, 
    peopleid INTEGER,
    score INTEGER
)
'''
cursor.execute(query)

In [12]:
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (1, 3, 7)")
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (2, 5, 8)")
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (2, 1, 9)")
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (1, 4, 8)")
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (2, 3, 7)")
cursor.execute("INSERT INTO quiz_results(quizzesid, peopleid, score) VALUES (1, 2, 8)")

In [13]:
rows = cursor.execute("SELECT * FROM quiz_results").fetchall()
print(rows)

[(1, 1, 3, 7), (2, 2, 5, 8), (3, 2, 1, 9), (4, 1, 4, 8), (5, 2, 3, 7), (6, 1, 2, 8)]


In [14]:
# find the firstname and lastname from the people table with id = 3
pid = 3
query = '''
SELECT firstname, lastname FROM people WHERE id = ?
'''
results = cursor.execute(query, (pid,)).fetchall()
print(results)

[('Daffy', 'Duck')]


In [15]:
# find the name of the quiz from the quizzes table with id = 2
qid = 2
query = '''
SELECT quizname FROM quizzes WHERE id = ?
'''
results = cursor.execute(query, (qid,)).fetchall()
print(results)

[('Tune',)]


In [16]:
# find the quiz name and number of items for quiz 2
qid = 2
query = '''
SELECT q.quizname, d.items 
FROM quizzes q, 
quiz_details d
WHERE q.id = ? 
AND q.id = d.quizzesid 
'''
results = cursor.execute(query, (qid,)).fetchall()
print(results)

[('Tune', 10)]


In [17]:
# find out how Bugs Bunny did on quiz 2
qid = 2
pid = 1
query = '''
SELECT q.quizname, p.firstname, p.lastname, r.score
FROM quizzes q,
people p,
quiz_results r
WHERE q.id = ?
AND p.id = ?
AND r.quizzesid = q.id
AND r.peopleid = p.id
'''
results = cursor.execute(query, (qid, pid,)).fetchall()
print(results)

[('Tune', 'Bugs', 'Bunny', 9)]


In [18]:
# find who took the Looney quiz, quiz name, first and last names, score, and year
qid = 1
query = '''
SELECT q.quizname, p.firstname, p.lastname, r.score, p.year 
FROM quizzes q,
people p,
quiz_results r
WHERE q.id = ?
AND r.quizzesid = q.id
AND r.peopleid = p.id
'''
results = cursor.execute(query, (qid,)).fetchall()
print(results)

[('Looney', 'Daffy', 'Duck', 7, 2), ('Looney', 'Foghorn', 'Leghorn', 8, 1), ('Looney', 'Fifi', 'La Fume', 8, 3)]


In [19]:
# using inner join, find who took the quizzes, quiz names, first and last names, scores, and years
query = '''
SELECT quizzes.quizname, people.firstname, people.lastname, quiz_results.score, people.year
FROM ((quiz_results
INNER JOIN people ON people.id = quiz_results.peopleid)
INNER JOIN quizzes ON quizzes.id = quiz_results.quizzesid)
ORDER BY quizzes.id
'''
results = cursor.execute(query).fetchall()
print(results)

[('Looney', 'Daffy', 'Duck', 7, 2), ('Looney', 'Foghorn', 'Leghorn', 8, 1), ('Looney', 'Fifi', 'La Fume', 8, 3), ('Tune', 'Tweety', 'Bird', 8, 4), ('Tune', 'Bugs', 'Bunny', 9, 1), ('Tune', 'Daffy', 'Duck', 7, 2)]


In [20]:
from contextlib import closing

with closing(sqlite3.connect("looneylms.db")) as connection:
    with closing(connection.cursor()) as cursor:
        rows = cursor.execute("SELECT 1").fetchall()
        print(rows)

[(1,)]


Start completely over

* Kernel Restart and clear output
* Delete looneylms.db 
* Kernel Restart and run all